In [ ]:
import wandb
import numpy as np
import pandas as pd

In [ ]:
api = wandb.Api()

In [ ]:
runs = api.runs(
    "tobifinn/test_diffusion_nextsim_regional",
    filters={
        "tags": "errors"
    }
)

In [ ]:
std = np.array([0.7506, 0.1848, 0.1968, 0.0836, 0.0878])

In [ ]:
columns = pd.MultiIndex.from_product([["1 it", "30 it"], ["SIT", "SIC", "damage", "SIU", "SIV", "nRMSE"]])
results = pd.DataFrame(columns=columns)

for run in runs:
    try:
        table = [artifact for artifact in run.logged_artifacts() if artifact.type == 'run_table'][0]
        table = table.get("test/scores.table.json")
        table = pd.DataFrame(table.data, columns=table.columns)
        table["nRMSE"] = np.sqrt(((table[["rmse_sit", "rmse_sic", "rmse_damage", "rmse_siu", "rmse_siv"]]**2)/std**2).mean(axis=1))
        table[["rmse_sit", "rmse_sic", "rmse_damage", "rmse_siu", "rmse_siv"]] /= std
        results.loc[run.name] = table[["rmse_sit", "rmse_sic", "rmse_damage", "rmse_siu", "rmse_siv", "nRMSE"]].iloc[[1, 30]].stack().values
    except IndexError:
        pass

In [ ]:
print(results.iloc[:, :6].round(2).to_latex(float_format="%.2f"))

In [ ]:
print(results.iloc[:, 6:].round(2).to_latex(float_format="%.2f"))